In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import preprocess_temp as P
import model.parsers as M

### Load Data

In [3]:
directory = './conala-corpus/'
train_file = directory + 'train.json'
test_file = directory + 'test.json'

with open(train_file) as f:
    train_data = json.load(f)
    
with open(test_file) as f:
    test_data = json.load(f)

### Let's preprocess the data. Everything is in Preprocess.py

In [4]:
# intent processing includes lowercase, remove punctuation'?'
train_intent, train_codes = P.process_data(train_data)
test_intent, test_codes = P.process_data(test_data)

In [5]:
# this class is used for code2actions and actions2code
ast_action = P.Ast_Action()

In [6]:
train_actions = []

for code in train_codes:
    train_actions.append(ast_action.code2actions(code))

In [7]:
word_lst = P.vocab_list(train_intent, cut_freq=5)
act_lst, token_lst = P.action_list(train_actions, cut_freq=5)

In [8]:
word2num = dict(zip(word_lst, range(0,len(word_lst))))
act2num = dict(zip(act_lst, range(0,len(act_lst))))
token2num = dict(zip(token_lst, range(0,len(token_lst))))

In [9]:
train_loader = P.get_train_loader(train_intent, train_actions, word2num, act2num, token2num)

In [10]:
test_loader = P.get_test_loader(test_intent, word2num)

In [11]:
action_index_copy = act2num[P.GenTokenAction('copy')]
action_index_gen = act2num[P.GenTokenAction('token')]

### Model

In [12]:
from collections import namedtuple
hyperParamMap = {
    #### General configuration ####
    'cuda': True,      # Use gpu
    'mode': 'train',   # train or test

    #### Embedding sizes ####
    'embed_size': 128,         # Size of word embeddings
    'action_embed_size': 128,  # Size of ApplyRule/GenToken action embeddings
    'field_embed_size': 64,    # Embedding size of ASDL fields
    'type_embed_size': 64,     # Embeddings ASDL types

    #### Decoding sizes ####
    'hidden_size': 256,        # Size of LSTM hidden states

    #### training schedule details ####
    'valid_metric': 'acc',                # Metric used for validation
    'valid_every_epoch': 1,               # Perform validation every x epoch
    'log_every': 10,                      # Log training statistics every n iterations
    'save_to': 'model',                   # Save trained model to
    'clip_grad': 5.,                      # Clip gradients
    'max_epoch': 10,                      # Maximum number of training epoches
    'optimizer': 'Adam',                  # optimizer
    'lr': 0.001,                          # Learning rate
    'lr_decay': 0.5,                      # decay learning rate if the validation performance drops
    'verbose': False,                     # Verbose mode

    #### decoding/validation/testing ####
    'load_model': None,                   # Load a pre-trained model
    'beam_size': 5,                       # Beam size for beam search
    'decode_max_time_step': 100,          # Maximum number of time steps used in decoding and sampling
    'sample_size': 5,                     # Sample size
    'test_file': '',                      # Path to the test file
    'save_decode_to': None,               # Save decoding results to file
}

HyperParams = namedtuple('HyperParams', list(hyperParamMap.keys()), verbose=False)
hyperParams = HyperParams(**hyperParamMap)

In [35]:
model = M.Model(hyperParams, action_size=len(act_lst), token_size=len(token_lst), word_size=len(word_lst), 
                      action_index_copy=action_index_copy, action_index_gen=action_index_gen)

In [36]:
import torch
import time

In [37]:
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)
lossFunc = torch.nn.CrossEntropyLoss()

In [38]:
epoch_begin = time.time()
for e in range(10):
    for batch_ind, x in enumerate(train_loader):
        optimizer.zero_grad()

        (action_logits, action_labels), (copy_logits, copy_labels), (token_logits, token_labels) = model(x)

        loss1 = lossFunc(action_logits, action_labels)
        loss2 = torch.DoubleTensor([0.0])
        if len(copy_logits) > 0:
            loss2 = lossFunc(copy_logits, copy_labels)
        loss3 = torch.DoubleTensor([0.0])
        if len(token_logits) > 0:
            loss3 = lossFunc(token_logits, token_labels)

        total_loss = loss1 + loss2.double() + loss3.double()
        total_loss.backward()

        # clip gradient
        if hyperParams.clip_grad > 0.:
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), hyperParams.clip_grad)

        optimizer.step()

        if batch_ind % hyperParams.log_every == hyperParams.log_every - 1:
            print("Action loss: {}".format(loss1.data))
            print("Copy loss: {}".format(loss2.data))
            print("Token loss: {}".format(loss3.data))
            print('-------------------------------------------------------')
            report_loss = report_examples = 0.

    print('epoch elapsed %ds' % (time.time() - epoch_begin))

Action loss: 2.016467615584445
Copy loss: 2.961787462234497
Token loss: 5.116165637969971
-------------------------------------------------------
Action loss: 1.6260644621086704
Copy loss: 2.511514902114868
Token loss: 4.443061351776123
-------------------------------------------------------
Action loss: 1.3185282024003342
Copy loss: 2.413918972015381
Token loss: 3.7202351093292236
-------------------------------------------------------
Action loss: 1.0786224066894838
Copy loss: 2.59718656539917
Token loss: 3.972409963607788
-------------------------------------------------------
Action loss: 1.4810984135296084
Copy loss: 2.2959377765655518
Token loss: 4.179492473602295
-------------------------------------------------------
Action loss: 1.4190842994179527
Copy loss: 2.313582181930542
Token loss: 4.215724945068359
-------------------------------------------------------
Action loss: 1.419256925725419
Copy loss: 2.427631378173828
Token loss: 4.175936698913574
----------------------------

Action loss: 0.5962390033726052
Copy loss: 1.9728648662567139
Token loss: 2.480133295059204
-------------------------------------------------------
epoch elapsed 720s
Action loss: 0.6383688236411884
Copy loss: 1.690181016921997
Token loss: 2.1056313514709473
-------------------------------------------------------
Action loss: 0.6901127979819148
Copy loss: 1.7206459045410156
Token loss: 2.44681715965271
-------------------------------------------------------
Action loss: 0.540737620880067
Copy loss: 1.8521512746810913
Token loss: 1.835858941078186
-------------------------------------------------------
Action loss: 0.6694366436070495
Copy loss: 1.6602026224136353
Token loss: 2.3602547645568848
-------------------------------------------------------
Action loss: 0.6037223597773073
Copy loss: 2.2827999591827393
Token loss: 1.8742506504058838
-------------------------------------------------------
Action loss: 0.5091142660736357
Copy loss: 2.2652297019958496
Token loss: 2.314922332763672
-

In [44]:
torch.save((model).state_dict(), 'Parameters/frist.t7')

In [45]:
model.load_state_dict(torch.load('Parameters/frist.t7'))